#### 1. 네이버 웹툰 추천하기???
1. 콘텐츠 기반 필터링 추천 시스템
    - 사용자가 특정한 아이템을 선호하는 경우, 그 아이템과 비슷한 아이템을 추천하는 방식

2. 최근접 이웃 협업 필터링
    - 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가
    - 사용자기반 : 당신과 비슷한 고객들이 다음 상품도 구매했음
  - 아이템 기반 : 이 상품을 선택한 다른 고객들은 다음 상품도 구매했음
  - 일반적으로는 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 더 높음
    - 비슷한 영화를 좋아한다고 취향이 비슷하다고 판단하기 어렵거나
    - 매우 유명한 영화는 취향과 관계없이 관람하는 경우가 많고
    - 사용자들이 평점을 매기지 않는 경우가 많기 때문
    
3. 잠재 요인 협업 필터링
    - 사용자-아이템 평점 행렬 데이터를 이용해서 '잠재요인'을 도출하는 것
    - 주요인과 아이템에 대한 잠재요인에 대해 행렬분해를 하고 다시 행렬곱을 통해 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성하는 것

In [76]:
# 데이터 읽기
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

# load file
df_naver = pd.read_csv('naver_webtoon_info.csv')
df_naver.tail()

,Unnamed: 0,title,genre,content,age
328,328,"그녀의 버킷리스트, 황양&솦","스토리, 드라마",미련없는 죽음을 위한그녀의 완벽한 버킷리스트,12세 이용가
329,329,"선녀야 야옹해봐!, 햄톨탱크","스토리, 로맨스",사랑 인연을 이어주기 위해 지상에 온 선녀 초미.톱스타 강수에게 냥줍을 당하다..!!,전체연령가
330,330,"갓!김치, 김민우","에피소드, 개그",​1000% 신토불이 히어로 등장!,전체연령가
331,331,"속삭이는 e로맨스, 최경아","스토리, 로맨스",불면증에 걸린 남자와 책만 읽으면 모두를 잠들게하는 여자가 만났다.,전체연령가
332,332,"호곡, 감대","스토리, 판타지",'이제 우리 차례야'조선의 왕 '천군'이 사라지고 3년 뒤...한 소녀가 나타나다.,12세 이용가


In [82]:
# title에서 제목과 작가 분리하기
title = df_naver['title'].str.split(',', n=1, expand=True)
df_naver['title'] = title[0]
df_naver['writer'] = title[1]
df_naver.tail()

In [86]:
# genre의 각 단어들을 하나의 문장(띄어쓰기로 구분된)으로 변환
# 장르를 , 기준으로 나눈 후 다시 합치기
df_naver['genres'] = df_naver['genre'].str.split(',')
df_naver['genres'] = df_naver['genres'].apply(lambda x: ('').join(x))
df_naver.tail()

,Unnamed: 0,title,genre,content,age,writer,genres
328,328,그녀의 버킷리스트,"스토리, 드라마",미련없는 죽음을 위한그녀의 완벽한 버킷리스트,12세 이용가,황양&솦,스토리 드라마
329,329,선녀야 야옹해봐!,"스토리, 로맨스",사랑 인연을 이어주기 위해 지상에 온 선녀 초미.톱스타 강수에게 냥줍을 당하다..!!,전체연령가,햄톨탱크,스토리 로맨스
330,330,갓!김치,"에피소드, 개그",​1000% 신토불이 히어로 등장!,전체연령가,김민우,에피소드 개그
331,331,속삭이는 e로맨스,"스토리, 로맨스",불면증에 걸린 남자와 책만 읽으면 모두를 잠들게하는 여자가 만났다.,전체연령가,최경아,스토리 로맨스
332,332,호곡,"스토리, 판타지",'이제 우리 차례야'조선의 왕 '천군'이 사라지고 3년 뒤...한 소녀가 나타나다.,12세 이용가,감대,스토리 판타지


In [87]:
# 문자열로 변환된 genres를 CountVectorize 수행
from sklearn.feature_extraction.text import CountVectorizer

# min_df : 0.0, 1.0 사이의 실수, 디폴트 1, 단어장에 포함되기 위한 최소 빈도
# ngram_range : (min_n, max_n) 튜플, n-gram 범위
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(df_naver['genres'])
print(genre_mat.shape)

(333, 36)


In [88]:
# 문장의 유사도 측정을 하는 방법 중 하나인 코사인 유사도 측정을 수행
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(333, 333)
[[1.         0.33333333 0.         0.33333333 0.33333333 0.
  0.33333333 0.         0.33333333 0.33333333 0.         0.33333333
  0.33333333 0.33333333 0.33333333 1.         0.33333333 1.
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.         0.         0.33333333 0.33333333 0.33333333 1.
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.
  0.33333333 0.         0.33333333 0.33333333 1.         0.33333333
  1.         0.33333333 0.         0.33333333 0.33333333 0.33333333
  1.         0.33333333 0.33333333 1.         0.33333333 1.
  0.33333333 0.33333333 0.         0.33333333 1.         1.
  0.         1.         0.33333333 0.33333333 0.         1.
  0.         0.33333333 0.33333333 1.         0.33333333 0.33333333
  0.33333333 0.         0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.33333333 0

In [89]:
# genre_sim 객체에서 높은 값 순으로 정렬
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[332 153 157 163 173 178 184 189 194 201 203 205 213 223 233 241 244 247
  260 276 281 154 152 286 150  15  17  35  46  48  54  57  59  64  65  67
   71  75 103 112 116 127 128 136 284   0 305 320 288 307 308 300 313  98
   99 100 102 287 317  97 105 106 107 110 111 104  94  95 113  93  92  91
   90  89  88  87  86  85  84  83 316 117 114 312 151 311 149 148 147 146
  145 142 141 140 139 138 137 135 115 134 133 132 131 314 126 123 122 121
  120 119 118  81  82  78  80  20 328  34  33  32  29  28  27  26  25  24
   23  22  21  19  37  18 329  16  14  13  12  11   9   8   6   4   3   1
   36  38  77 324  76 318  74  73  69  68 321 322  63  61  60 323  58  56
   39  55 325  53  52  51  49 326  47 327  45  44  42  40 310 143 222 230
  248 292 293 243 242 239 237 236 235 234 295 232 231 229 209 228 227 226
  224 220 219 218 216 215 214 297 212 211 249 250 251 252 283 289 280 279
  278 277 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 291
  259 258 256 255 254 210 166 208 192 

In [90]:
# 추천 웹툰을 DataFrame으로 반환하는 함수
def find_sim_webtoon(df, sorted_ind, title_name, top_n=10):
    title_webtoon = df_naver[df_naver['title'] == title_name]
    
    title_index = title_webtoon.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [92]:
title_webtoon = find_sim_webtoon(df_naver, genre_sim_sorted_ind, '그녀의 버킷리스트', 10)
title_webtoon[['title', 'content', 'genre']]

[[166 251 122 126  69 198 262 263 265 196]]


,title,content,genre
166,킬더킹,"왕의 재능이란 무엇일까?왕좌의 새로운 주인, 진정한 왕에 걸맞는 재능을 가려내는 게임이 시작된다!","스토리, 드라마"
251,나를 바꿔줘,인생을 바꾸기 위해 얼굴을 훔치다,"스토리, 드라마"
122,정순애 식당,"미각을 잃은 사람도 이곳에 오면 ""맛있다""를 외칩니다!!따뜻한 집밥과 사람 사는 냄새가 나는 힐링공간 정순애식당에 어서오세요~","스토리, 드라마"
126,오라존미,만화를 그리는 작가와 만화를 말하는 작가.,"스토리, 드라마"
69,견우와 선녀,죽음이 가까워진 사람들은 '거꾸로' 걸어들어 온다그리고 그게 하필 내 이상형이라니!?,"스토리, 드라마"
198,1초,구조율 100%의 전설적인 소방관​그의 특별한 능력은 무엇인가?,"스토리, 드라마"
262,고인의 명복,"이승과 저승, 삶과 죽음을 둘러싼한국형 미스터리 판타지","스토리, 드라마"
263,왕세자 입학도,"조선 후기, 성균관에 자그마한 신참이 들어온다! 모종의 이유로 정체를 숨기고 성균관 유생이 된 왕세자가 보고 겪는 체험! 민중의 현장!","스토리, 드라마"
265,누군가의 로섬,"로섬이란 남자, 정체가 궁금하다.","스토리, 드라마"
196,닥터 프로스트 시즌 3~4,세상으로부터의 무력감. 엄습하는 두려움. 또다시 덧씌운 분노.끝없는 혐오의 고리로 만들어진 지옥도가 펼쳐진 대한민국.닥터프로스트와 그의 제자 윤성아가 거대한 음모를 막아선다.,"스토리, 드라마"
